In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Conv1DClassifier(nn.Module):
    def __init__(self, initial_length, num_classes):
        super(Conv1DClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.pool5 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv6 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=2)
        self.pool6 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Calculate the size of the feature map after all convolution and pooling layers
        # Input size is 65000
        final_length = initial_length // (2**6)  # After six max-pooling layers, each reducing size by half

        self.fc1 = nn.Linear(512 * final_length, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))
        x = self.pool4(self.relu(self.conv4(x)))
        x = self.pool5(self.relu(self.conv5(x)))
        x = self.pool6(self.relu(self.conv6(x)))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
8001536 // 512

15628

In [4]:
2**6

64

In [5]:
# Generate a synthetic dataset (for demonstration purposes)
def generate_synthetic_data(num_samples, sequence_length, num_classes):
    data = np.load('ecgs_labels.npy')
    X = torch.FloatTensor(data[:,:-1]).unsqueeze(1)
    y = torch.LongTensor(data[:,-1])
    # X = torch.randn(num_samples, 1, sequence_length)
    # y = torch.randint(0, num_classes, (num_samples,))
    return X, y

def generate_data(seed=42):
    data = np.load('ecgs_labels.npy')
    X = torch.FloatTensor(data[:,:-1]).unsqueeze(1)
    y = torch.LongTensor(data[:,-1])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    # X = torch.randn(num_samples, 1, sequence_length)
    # y = torch.randint(0, num_classes, (num_samples,))
    return torch.FloatTensor(X_train), torch.FloatTensor(X_test), torch.LongTensor(y_train), torch.LongTensor(y_test)

def generate_data2(seed=42):
    data = np.load('ecgs_labels.npy')
    X = torch.FloatTensor(np.load('hrv_signals.npy')).unsqueeze(1)
    y = torch.LongTensor(np.load('hrv_labels.npy'))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    # X = torch.randn(num_samples, 1, sequence_length)
    # y = torch.randint(0, num_classes, (num_samples,))
    return torch.FloatTensor(X_train), torch.FloatTensor(X_test), torch.LongTensor(y_train), torch.LongTensor(y_test)

# Generate data
# X, y = generate_synthetic_data(num_samples, sequence_length, num_classes)
X_train, X_test, y_train, y_test = generate_data(np.random.randint(900))
print(X_train.shape)


# print(X.shape)
# print(X.dtype)
# Initialize the model, loss function, and optimizer


torch.Size([932, 1, 65000])


In [6]:
hrv_data = np.load('better_hrv_data.npy')
hrvs, y = hrv_data[:,:-1], hrv_data[:,-1]
hrvs.shape

(400, 25000)

In [7]:
ecg_data = np.load('better_ecg_data.npy')
ecgs, y = ecg_data[:,:-1], ecg_data[:,-1]
ecgs.shape

(400, 50000)

In [8]:
X = ecg_data # balanced_hrvs

In [9]:
seed = 42

In [10]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(scaler.fit_transform(X), y, test_size=0.2, random_state=seed)

In [11]:
ecg_data_train = np.load('train_ecg_data.npy')
X_train, y_train = ecg_data_train[:,:-1], ecg_data_train[:,-1]
ecg_data_test= np.load('test_ecg_data.npy')
X_test, y_test = ecg_data_test[:,:-1], ecg_data_test[:,-1]

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [17]:
hrv_data_train = np.load('train_hrv_data.npy')
X_train, y_train = hrv_data_train[:,:-1], hrv_data_train[:,-1]
hrv_data_test= np.load('test_hrv_data.npy')
X_test, y_test = hrv_data_test[:,:-1], hrv_data_test[:,-1]

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [18]:
# Parameters
num_classes = len(np.unique(y_train))
sequence_length = X_train.shape[-1]  # Length of each input sequence
# num_samples = 1000
batch_size = 8
num_epochs = 200
learning_rate = 0.001


In [19]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [20]:
X_train, X_test = torch.FloatTensor(X_train), torch.FloatTensor(X_test)
y_train, y_test = torch.LongTensor(y_train), torch.LongTensor(y_test)

dataset = TensorDataset(X_train.unsqueeze(1), y_train) 
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_test = TensorDataset(X_test.unsqueeze(1), y_test) 
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [21]:
model = Conv1DClassifier(initial_length=sequence_length, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # print(inputs.shape)
        # print(labels.shape)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    if (epoch+1) % 1 == 0 :
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Train Accuracy: {correct / total:.2%}')

            correct = 0
            total = 0
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Test Accuracy: {correct / total:.2%}')

        

# Evaluation (for simplicity, using the same synthetic dataset)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.2%}')


Epoch [1/200], Loss: 0.8197
Train Accuracy: 50.25%
Test Accuracy: 50.00%
Epoch [2/200], Loss: 0.7017
Train Accuracy: 50.00%
Test Accuracy: 50.00%
Epoch [3/200], Loss: 0.6654
Train Accuracy: 55.25%
Test Accuracy: 57.50%
Epoch [4/200], Loss: 0.6308
Train Accuracy: 68.25%
Test Accuracy: 71.00%
Epoch [5/200], Loss: 0.6743
Train Accuracy: 60.50%
Test Accuracy: 61.00%
Epoch [6/200], Loss: 0.7308
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [7/200], Loss: 0.6920
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [8/200], Loss: 0.6888
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [9/200], Loss: 0.6909
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [10/200], Loss: 0.6906
Train Accuracy: 50.50%
Test Accuracy: 50.50%
Epoch [11/200], Loss: 0.6873
Train Accuracy: 50.25%
Test Accuracy: 50.00%
Epoch [12/200], Loss: 0.6896
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [13/200], Loss: 0.6889
Train Accuracy: 50.50%
Test Accuracy: 50.00%
Epoch [14/200], Loss: 0.6884
Train Accuracy: 50

KeyboardInterrupt: 